In [15]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['font.serif'] = ['SimHei']

import seaborn as sns
%matplotlib inline
p = sns.color_palette()
sns.set_style("darkgrid",{"font.sans-serif":['simhei', 'Arial']})

# 航班数据
feature = pd.read_table('/Users/tuyu/000000毕业设计/Flight_Delay/Train_Feature/final_feature.csv',sep=',',encoding='gb2312')
feature.head()

,航班编号,计划起飞时间,计划到达时间,计划起飞时刻,计划到达时刻,计划飞行时间,起飞间隔,前序延误,平均延误时间,最大延误时间,...,本时段实际进港延误航班数,本时段离港延误时长,本时段离港平均延误时长,本时段离港延误率,上一时段离港平均延误时长,上一时段进港延误时长,上一时段离港延误时长,上一时段实际进港航班数,上一时段进港延误率,标签
0,CZ3855,16801,16801,0,3,3.666667,NaN,NaN,1.963379,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,CZ3855,16801,16801,0,3,3.666667,NaN,NaN,1.963379,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,CZ3855,16801,16801,0,1,1.250000,NaN,NaN,1.963379,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,CZ3855,16801,16801,0,1,1.250000,NaN,NaN,1.963379,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,CZ3295,16801,16801,0,1,1.166667,NaN,NaN,0.324175,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [16]:
feature0 = feature[feature['标签'] == 0]
feature0new = feature0.sample(frac=0.1, replace=False, random_state=None,axis=0)
feature1 = feature[feature['标签'] == 1]
featureGBDT = pd.concat([feature0new,feature1],axis=0, ignore_index=True)

In [17]:
feature0LSTM = feature0new.sample(frac=0.12, replace=False, random_state=None,axis=0)
feature1LSTM = feature1.sample(frac=0.1, replace=False, random_state=None,axis=0)

featureLSTM = pd.concat([feature0LSTM,feature1LSTM],axis=0, ignore_index=True)

In [18]:
print(len(featureLSTM))
print(sum(featureLSTM['标签'] == 1))
print(sum(featureLSTM['标签'] == 0))

140294
55262
85032


In [19]:
del(featureLSTM['本时段离港平均延误时长'])
del(featureLSTM['上一时段离港平均延误时长'])
# 1.前序延误 = 前序航班 实际到达时间 - 计划到达时间
# 2.起飞间隔 = 当前航班的计划起飞时间 - 前序航班实际到达时间
featureLSTM['起飞间隔'] = featureLSTM['起飞间隔'].fillna(0)
featureLSTM['前序延误'] = featureLSTM['前序延误'].fillna(0)
featureLSTM['出发机场天气得分'] = featureLSTM['出发机场天气得分'].interpolate()
featureLSTM['到达机场天气得分'] = featureLSTM['到达机场天气得分'].interpolate()
featureLSTM['平均延误时间'] = featureLSTM['平均延误时间'].interpolate()
featureLSTM['最大延误时间'] = featureLSTM['最大延误时间'].interpolate()
featureLSTM['延误时间中位数'] = featureLSTM['延误时间中位数'].interpolate()
featureLSTM['延误时间标准差'] = featureLSTM['延误时间标准差'].interpolate()
featureLSTM['本时段计划离港航班数'] = featureLSTM['本时段计划离港航班数'].fillna(featureLSTM['本时段计划离港航班数'].mean())
featureLSTM['本时段实际离港航班数'] = featureLSTM['本时段实际离港航班数'].fillna(featureLSTM['本时段实际离港航班数'].mean())
featureLSTM['本时段进港延误时长'] = featureLSTM['本时段进港延误时长'].fillna(featureLSTM['本时段进港延误时长'].mean())
featureLSTM['本时段实际进港延误航班数'] = featureLSTM['本时段实际进港延误航班数'].fillna(featureLSTM['本时段实际进港延误航班数'].mean())
featureLSTM['本时段离港延误时长'] = featureLSTM['本时段离港延误时长'].fillna(featureLSTM['本时段离港延误时长'].mean())
featureLSTM['本时段离港延误率'] = featureLSTM['本时段离港延误率'].fillna(featureLSTM['本时段离港延误率'].mean())
featureLSTM['上一时段进港延误时长'] = featureLSTM['上一时段进港延误时长'].fillna(featureLSTM['上一时段进港延误时长'].mean())
featureLSTM['上一时段离港延误时长'] = featureLSTM['上一时段离港延误时长'].fillna(featureLSTM['上一时段离港延误时长'].mean())
featureLSTM['上一时段实际进港航班数'] = featureLSTM['上一时段实际进港航班数'].fillna(featureLSTM['上一时段实际进港航班数'].mean())
featureLSTM['上一时段进港延误率'] = featureLSTM['上一时段进港延误率'].fillna(featureLSTM['上一时段进港延误率'].mean())


In [20]:
featureLSTM

,航班编号,计划起飞时间,计划到达时间,计划起飞时刻,计划到达时刻,计划飞行时间,起飞间隔,前序延误,平均延误时间,最大延误时间,...,本时段实际离港航班数,本时段进港延误时长,本时段实际进港延误航班数,本时段离港延误时长,本时段离港延误率,上一时段进港延误时长,上一时段离港延误时长,上一时段实际进港航班数,上一时段进港延误率,标签
0,9C8823,16948,16948,2,5,2.833333,1.383333,-0.300000,0.618392,10.000000,...,15.048232,0.766667,0.000000,0.533333,0.031895,-0.800000,0.250000,27.000000,0.035714,0
1,CZ3213,16976,16976,4,5,1.416667,9.216667,-0.216667,0.563878,10.000000,...,26.000000,0.416667,0.000000,10.133333,0.000000,15.100000,10.500000,3.000000,0.000000,0
2,MF8059,16925,16925,3,5,2.083333,1.083333,-0.250000,0.409289,6.266667,...,15.048232,-0.716667,0.374075,-11.106392,0.031895,-13.856496,-1.109715,15.527144,0.027644,0
3,SC4927,17212,17212,3,5,2.750000,1.066667,-0.233333,0.785284,10.000000,...,2.000000,1.133333,0.000000,0.066667,0.000000,-2.883333,0.300000,7.000000,0.125000,0
4,CZ6732,17296,17296,8,9,1.666667,0.966667,0.116667,0.729914,10.000000,...,24.000000,4.316667,0.000000,23.966667,0.040000,15.233333,20.050000,11.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140289,ZH9386,17289,17289,1,4,2.750000,1.200000,-0.116667,0.653060,10.883333,...,15.048232,-2.036127,1.000000,-11.106392,0.031895,-13.856496,-1.109715,15.527144,0.027644,1
140290,MU5411,17166,17166,13,16,3.583333,-1.300000,-0.533333,10.000000,10.000000,...,12.000000,-5.816667,2.000000,4.566667,0.076923,-3.316667,7.450000,14.000000,0.000000,1
140291,NS3228,16932,16932,11,12,1.000000,0.000000,10.000000,1.700815,10.000000,...,8.000000,0.100000,0.000000,1.533333,0.000000,0.883333,2.166667,4.000000,0.000000,1
140292,HU7089,17176,17176,9,11,2.500000,1.150000,-0.233333,1.012635,10.166667,...,13.000000,3.266667,0.000000,12.883333,0.071429,4.683333,14.650000,15.000000,0.062500,1


In [21]:
# featureLSTM = featureLSTM.sample(frac=0.1, replace=False, random_state=None,axis=0)

In [22]:
import pandas as pd 
import numpy as np
from numpy import *
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

dataset = featureLSTM.as_matrix()
X = dataset[:,1:36]
Y = dataset[:,36]
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.3, random_state=7)

x_train = (x_train - x_train.min(axis=0)) / (x_train.max(axis=0) - x_train.min(axis=0))
x_test = (x_test - x_test.min(axis=0)) / (x_test.max(axis=0) - x_test.min(axis=0))


/Users/tuyu/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

clf = SVC(kernel='rbf', C=200, degree=5,  gamma=0.2,
    max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
clf.fit(x_train,y_train.astype('int'))
from sklearn.metrics import accuracy_score
#KS定义函数
from sklearn import metrics
from sklearn.metrics import roc_auc_score                                                                                        
def ks(y_predicted, y_true):
    label = y_true
    #label = y_true.get_label()
    fpr,tpr,thres = metrics.roc_curve(label,y_predicted,pos_label=1)
    return 'ks',abs(fpr - tpr).max()
#模型预测
y_pred = clf.predict(x_test)
#测试一下精度
predictions =[round(value) for value in y_pred]
y_test= y_test.tolist()
y_test = np.array([i for i in y_test])
accuracy =accuracy_score(y_test,y_pred)
print("准确率: %.2f%%" % (accuracy*100.0))
#测试一下KS值，不建议写到论文中，高的有些离谱
print("KS的值为", ks(y_pred, y_test))
#测试一下AUC的值
print("AUC的值为:", roc_auc_score(y_pred, y_test))
#混淆矩阵等具体模型结果
from sklearn.metrics import classification_report 
print(classification_report(y_pred, y_test))

In [ ]:
prediction_test_predprob = clf.predict_proba(x_test)
prediction_test_predprob

In [ ]:
import pickle
f= open('/Users/tuyu/000000毕业设计/Flight_Delay/Model/model/svm_model.pickle','wb')
pickle.dump(clf,f)
f.close()

